In [1]:
import os

class Hinter:
    '''
    Hinter is used to load a dictionary and obtain some suggestions
    regarding the next possible letters or compatible words.
    '''
    
    def __init__(self, words):
        self.words = words

 
    def load_english_dict():
        '''
        Loads the english dictionary and returns a Hinter object with
        the words loaded into the self.words list
        '''
        ENGLISH_FILENAME = "dict" + os.sep + "english.txt"
        words = [i.replace("\n","") for i in open(ENGLISH_FILENAME)]
        return Hinter(words)

    def compatible_words(self, word, limit = 10):
        '''
        Returns the words that starts with the "word" parameter.
        The "limit" parameter defines how many words the function
        returns at maximum
        '''
        output = []
        word_count = 0
        #Cycle through all the words to find the ones that starts with "word"
        for i in self.words:
            if i.startswith(word):
                output.append(i)
                word_count+=1
            if word_count>=limit: #If limit is reached, exit
                break
        return output

    def next_letters(self, word):
        '''
        Returns a list of compatible letters.
        A letter is compatible when there are words that starts with "word"
        and are followed by the letter.
        '''
        #Get 100 compatible words
        words = self.compatible_words(word, 100)
        letters = []
        #Cycle through all the compatible words
        for i in words:
            if len(i)>len(word): #if the "word" is longer than a compatible word, skip
                letter = i[len(word):len(word)+1] #Get the following letter
                if not letter in letters: #Avoid duplicates
                    letters.append(letter)
        return letters

    def does_word_exists(self, word):
        '''
        Check if a specific word exists in the loaded dictionary
        '''
        if word in self.words:
            return True
        else:
            return False

    def most_probable_letter(self, clf, classes, linearized_sample, word):
        '''
        Get the most probable letter for a given recorded sign and the current word
        '''
        if word=="":
            return None
        
        probabilities = clf.predict_log_proba(linearized_sample)
        ordered = sorted(classes)
        values = {}
        for i in xrange(len(probabilities[0])):
            values[round(probabilities[0,i], 5)] = classes[ordered[i]]
        ordered = sorted(values, reverse=True)
        possible_letters = self.next_letters(word)
        for i in ordered:
            if values[i] in possible_letters:
                return values[i]
        return None


In [2]:
import time 
import serial
ser = serial.Serial('COM5', 38400, timeout = 0)

while 1:
    try:
        print ser.readline()
        time.sleep(1)
    except ser.SerialTimeoutException:
        print('Data could not be read')
        time.sleep(1)                                                                 






STARTING BATCH

START -1884 -588 17496 -19 -26 -162 END

START 3132 -1136 12044 -479 -1580 -68 END

START -452 -988 15088 -293 -486 -180 END

START -2840 -572 19244 -279 -502 -156 END

START -2088 -568 17736 -126 -132 -126 END

START -1892 -708 17256 -133 -64 -145 END

START -1780 -744 17216 -141 -112 -136 END



AttributeError: 'Serial' object has no attribute 'SerialTimeoutException'

In [3]:

import serial, os, signals, sys
from sklearn.externals import joblib

def print_sentence_with_pointer(sentence, position):
    print (sentence)
    print (" "*position + "^")

#Sentence used to get samples because it contains all letters.
#ALTERNATIVE: the quick brown fox jumps over the lazy dog
test_sentence = "pack my box with five dozen liquor jugs"

#Mode parameters, controlled using sys.argv by the terminal
TRY_TO_PREDICT = False
SAVE_NEW_SAMPLES = False
FULL_CYCLE = False
ENABLE_WRITE = True
TARGET_ALL_MODE = False
AUTOCORRECT = True
DELETE_ALL_ENABLED = False

#Serial parameters
SERIAL_PORT = "COM5"
BAUD_RATE = 384000
TIMEOUT = 100

#Recording parameters
target_sign = "a"
current_batch = "0"
target_directory = "data"

current_test_index = 0

#Analyzes the arguments to enable a specific mode

arguments = {}

for i in sys.argv[1:]:
    if "=" in i:
        sub_args = i.split("=")
        arguments[sub_args[0]]=sub_args[1]
    else:
        arguments[i]=None

#If there are arguments, analyzes them
if len(sys.argv)>1:
    if arguments.has_key("target"):
        target_sign = arguments["target"].split(":")[0]
        current_batch = arguments["target"].split(":")[1]
        print ("TARGET SIGN: '{sign}' USING BATCH: {batch}".format(sign=target_sign, batch = current_batch))
        SAVE_NEW_SAMPLES = True
    if arguments.has_key("predict"):
        TRY_TO_PREDICT = True
    if arguments.has_key("write"):
        TRY_TO_PREDICT = True
        ENABLE_WRITE = True
    if arguments.has_key("test"):
        current_batch = arguments["test"]
        TARGET_ALL_MODE = True
        SAVE_NEW_SAMPLES = True
    if arguments.has_key("noautocorrect"):
        AUTOCORRECT=False
    if arguments.has_key("port"):
        SERIAL_PORT = arguments["port"]

clf = None
classes = None
sentence = ""


#Loads the machine learning model from file
if TRY_TO_PREDICT:
    print ("Loading model...")
    clf = joblib.load('model.pkl')
    classes = joblib.load('classes.pkl')


print ("OPENING SERIAL_PORT '{port}' WITH BAUDRATE {baud}...".format(port = SERIAL_PORT, baud = BAUD_RATE))

print ("IMPORTANT!")
print ("To end the program hold Ctrl+C and send some data over serial")

#Opens the serial port specified by SERIAL_PORT with the specified BAUD_RATE
ser = serial.Serial(SERIAL_PORT, BAUD_RATE, timeout = TIMEOUT)

output = []

in_loop = True
is_recording = False

current_sample = 0


    
#Resets the output file
output_file = open("output.txt","w")
output_file.write("")
output_file.close()

#If TARGET_ALL_MODE = True, print the sentence with the current position
if TARGET_ALL_MODE:
    print_sentence_with_pointer(test_sentence, 0)

try:
    while in_loop:
        #Read a line over serial and deletes the line terminators
        line = ser.readline().replace("\r\n","")
        #If it receive "STARTING BATCH" it starts the recording
        if line=="STARTING BATCH":
        #Enable the recording
            is_recording = True
            #Reset the buffer
            output = []
            print ("RECORDING...")
        elif line=="CLOSING BATCH": #Stops recording and analyzes the result
            #Disable recording
            is_recording = False
            if len(output)>1: #If less than 1, it means error
                print ("DONE, SAVING...")

                #If TARGET_ALL_MODE is enabled changes the target sign
                    #according to the position
                if TARGET_ALL_MODE:
                    if current_test_index<len(test_sentence):
                        target_sign = test_sentence[current_test_index]
                    else:
                        #At the end of the sentence, it quits
                        print ("Target All Ended!")
                        quit()

                #Generates the filename based on the target sign, batch and progressive number
                filename = "{sign}_sample_{batch}_{number}.txt".format(sign = target_sign, batch = current_batch, number = current_sample)
                #Generates the path
                path = target_directory + os.sep + filename

                #If SAVE_NEW_SAMPLES is False, it saves the recording to a temporary file
                if SAVE_NEW_SAMPLES == False:
                    path = "tmp.txt"
                    filename = "tmp.txt"

                #Saves the recording in a file
                f = open(path, "w")
                f.write('\n'.join(output))
                f.close()
                print ("SAVED IN {filename}".format(filename = filename))

                current_sample += 1

                #If TRY_TO_PREDICT is True, it utilizes the model to predict the recording
                if TRY_TO_PREDICT:
                    print ("PREDICTING...")
                    #It loads the recording as a Sample object
                    sample_test = signals.Sample.load_from_file(path)

                    linearized_sample = sample_test.get_linearized(reshape=True)
                    #Predict the number with the machine learning model
                    number = clf.predict(linearized_sample)
                    #Convert it to a char
                    char = chr(ord('a')+number[0])

                    #Get the last word in the sentence
                    last_word = sentence.split(" ")[-1:][0]

                    #If AUTOCORRECT is True, the cross-calculated char will override the predicted one
                    if AUTOCORRECT and char.islower():
                        predicted_char =Hinter.load_english_dict.most_probable_letter(clf, classes, linearized_sample, last_word)
                        if predicted_char is not None:
                            print ("CURRENT WORD: {word}, PREDICTED {old}, CROSS_CALCULATED {new}".format(word = last_word, old = char, new = predicted_char))
                            char = predicted_char

                    #If the mode is WRITE, assigns special meanings to some characters
                    #and builds a sentence with each char
                    if ENABLE_WRITE:
                        if char == 'D': #Delete the last character
                            sentence = sentence[:-1]
                        elif char == 'A': #Delete all characters
                            if DELETE_ALL_ENABLED:
                                sentence = ""
                            else:
                                print ("DELETE_ALL_ENABLED = FALSE")
                        else: #Add the char to the sentence
                            sentence += char
                        #Prints the last char and the sentence
                        print ("[{char}] -> {sentence}".format(char = char, sentence = sentence))
                        #Saves the output to a file
                        output_file = open("output.txt","w")
                        output_file.write(sentence)
                        output_file.close()
                    else:
                        print (char)
            else: #In case of a corrupted sequence
                print ("ERROR...")
                current_test_index -= 1

            #If TARGET_ALL_MODE=True it shows the current position in the sentence
            if TARGET_ALL_MODE:
                current_test_index += 1
                print_sentence_with_pointer(test_sentence, current_test_index)
        else:
            #Append the current signal line in the recording
            output.append(line)
except KeyboardInterrupt: #When Ctrl+C is pressed, the loop terminates
    print ('CLOSED LOOP!')

#Closes the serial port
ser.close()



OPENING SERIAL_PORT 'COM5' WITH BAUDRATE 384000...
IMPORTANT!
To end the program hold Ctrl+C and send some data over serial
CLOSED LOOP!
